In [ ]:
import pandas as pd
# pd maximum row number
pd.set_option('display.max_columns', 500)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from copy import copy


In [ ]:
def fetch_table(url, i, driver):
    data = []

    driver.get(url)
    
    try:
        table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/main[1]/article/table"))
        )

        print("Found Table")

        header_row = table.find_element(By.TAG_NAME, "thead")
        headers = [th.text.strip() for th in header_row.find_elements(By.TAG_NAME, "th")]

        rows = table.find_elements(By.TAG_NAME, "tr")
        
        
        for row in rows:
            cols = row.find_elements(By.XPATH, ".//th | .//td")
            if cols:
                data.append([col.text.strip() for col in cols])

        print("Extracted Data:" , i, "~",  i + 60)

        df = pd.DataFrame(data=data, columns=headers)
    
    except Exception as e:
        print("Table not found from", i)
        return False
        
    return df

In [ ]:
url = "https://sofifa.com/players?type=all&ct%5B0%5D=2&lg%5B0%5D=13&lg%5B1%5D=31&lg%5B2%5D=53&lg%5B3%5D=19&lg%5B4%5D=16&showCol%5B0%5D=pi&showCol%5B1%5D=ae&showCol%5B2%5D=hi&showCol%5B3%5D=wi&showCol%5B4%5D=pf&showCol%5B5%5D=oa&showCol%5B6%5D=pt&showCol%5B7%5D=cr&showCol%5B8%5D=fi&showCol%5B9%5D=he&showCol%5B10%5D=sh&showCol%5B11%5D=vo&showCol%5B12%5D=dr&showCol%5B13%5D=cu&showCol%5B14%5D=fr&showCol%5B15%5D=lo&showCol%5B16%5D=bl&showCol%5B17%5D=ac&showCol%5B18%5D=sp&showCol%5B19%5D=ag&showCol%5B20%5D=re&showCol%5B21%5D=ba&showCol%5B22%5D=so&showCol%5B23%5D=ju&showCol%5B24%5D=st&showCol%5B25%5D=sr&showCol%5B26%5D=ln&showCol%5B27%5D=ar&showCol%5B28%5D=po&showCol%5B29%5D=vi&showCol%5B30%5D=cm&showCol%5B31%5D=ma&showCol%5B32%5D=sa&showCol%5B33%5D=sl&showCol%5B34%5D=aw&showCol%5B35%5D=dw&offset="



In [ ]:
service = Service(executable_path=ChromeDriverManager().install())
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(60)

total_df = pd.DataFrame()
i = 0

while True:
    url_offset = url + str(i)
    df = fetch_table(url_offset, i, driver)
    if df is False:
        print("Finished, no more data to fetch.")
        break
    
    total_df = pd.concat([total_df, df], ignore_index=True, axis=0)
    time.sleep(1.5)